In [ ]:
import argparse
import os
import numpy as np
import tqdm
import pickle

from matplotlib import cm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from data.dataloader import ModelNet_aligned
from model.DeepLatent_naive import *
from model.networks import *
from utils.utils import *

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='0'

checkpoint_dir = 'results/'
root = '/home/mmvc/mmvc-ny-nas/Yi_Shi/data/'
save_name = os.path.join(checkpoint_dir,'model_best')


# load latent_space
z_space = torch.load(save_name+'_latents.pt',map_location='cpu')
z_space_np = z_space.cpu().detach().numpy()
print(z_space_np.shape)

# load latent_objects (just test latent space)
z_obj = torch.load(save_name+'_test_latents.pt',map_location='cpu')
z_obj_np = z_obj.cpu().detach().numpy()
print(z_obj_np.shape)


In [ ]:
def read_cate_name(mode='train'):
    dataset = ModelNet_aligned(root,None,mode=mode,downsample_num=2048)
    indices = dataset.indices
    categ = dataset.categ
    names = dataset.npys_gen
    names_instance = [os.path.split(i)[-1].split('.')[0] for i in names]
    categs_instance = [categ.index('_'.join(i.split('_')[:-1])) for i in names_instance]
    return names_instance,categs_instance

def index2hot(indeces,rang=40):
    num_ind = len(indeces)
    b = np.zeros((num_ind,rang))
    b[np.arange(num_ind), indeces] = 1
    return b
    
names_instance,categs_instance = read_cate_name(mode='train')
print(len(categs_instance),len(names_instance))
y_test = index2hot(categs_instance)
print(np.shape(y_test))

In [5]:
def average_precision(ranked_relevant, num_relevant_retrievable):
    """Calculate average precision for a ranked set
    ranked_relevant : np.array
        ranked boolean array for whether the rank contained a relevant result
    num_relevant_retrievable : int
        number of possible relevant documents
    Returns
    -------
    average_precision : np.float64
    """
    # The rank will be used to divide to get precision for everything
    # to that point
    # based on: http://web.cecs.pdx.edu/~maier/cs510iri/IR_Lectures/CS_510iri_Lecture8RelevanceEvaluation-revised.pdf
    # This is a tricky calculation to be honest
    
    rank = np.arange(ranked_relevant.shape[0]) + 1
    indices = np.arange(ranked_relevant.shape[0])
    relevant_indices = np.zeros(ranked_relevant.shape[0])
    relevant_indices[indices[ranked_relevant]] = \
        np.cumsum(ranked_relevant[indices[ranked_relevant]])
    return np.sum(relevant_indices/rank)/num_relevant_retrievable

def query_latent_space(latent_object, latent_space, query_size=10):
    """Query a latent space (bottleneck features from NN)
    latent_object : np.array
        object as represented in an N-D space
    latent_space : np.array
        matrix of N-d latent vectors each representing a model
        in your associated latent space
    query_size : int
        number of relevant items to retrieve
    Returns
    -------
    top_n_sorted_sims : np.array
        cosine similarities for the top n rows in latent_space
        where n is specified by `query_size`
    top_n_sorted_indices : np.array
        indices of top n rows from the latent_space
    """
    sims = latent_space.dot(latent_object.T)
    sorted_sims_indices = np.argpartition(sims,
                                          range(-query_size, 0),
                                          axis=0)
    top_n_sorted_indices = sorted_sims_indices[:-query_size-1:-1].ravel()
    top_n_sorted_sims = sims[top_n_sorted_indices].ravel()
    return top_n_sorted_sims, top_n_sorted_indices



def _get_average_precisions(latent_space, x_latent_test, y_test):
    test_size = x_latent_test.shape[0]
    average_precisions = np.zeros(test_size)
    for i in range(test_size):
        
        num = i
        num_retrievable = (np.argmax(y_test[num]) == \
                               np.argmax(y_test, axis=1)).sum()
        # latent_object = latent_model.predict(x_test[num:num+1])
        latent_object = latent_space[num: num+1]
        sims, latent_indices = query_latent_space(latent_object,
                                                  latent_space,
                                                  test_size)
        ranked_relevant = np.argmax(y_test[num]) ==\
                            np.argmax(y_test[latent_indices], axis=1)

        average_precisions[i] = average_precision(ranked_relevant, num_retrievable)
        if i % 100 == 0:
            print('precisions_done_calculating_{}'.format(i))
            print(ranked_relevant[:10])
            print(num_retrievable)
    return average_precisions



In [6]:
average_precisions = _get_average_precisions(z_space_np, z_space_np, y_test)
print(average_precisions)

precisions_done_calculating_0
[ True False  True False False False False False False False]
622
precisions_done_calculating_100
[ True False False False False  True False  True False False]
622
precisions_done_calculating_200
[ True False False False False False False False False False]
622
precisions_done_calculating_300
[ True False  True False False False False False False False]
622
precisions_done_calculating_400
[ True  True False False False False False False False False]
622
precisions_done_calculating_500
[ True False  True False  True False False False False False]
622
precisions_done_calculating_600
[ True  True False False False False False False False False]
622
precisions_done_calculating_700
[ True False False False False False False False False False]
301
precisions_done_calculating_800
[ True False False False False False False False False False]
301
precisions_done_calculating_900
[ True False False False False False False False False False]
301
precisions_done_calcul